In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

# warningsを非表示にする
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
y_col = "MIS_Status"

cat_col = [
    "RevLineCr",
    "LowDoc",
    "Sector",
    "State",
    "BankState",
    "FranchiseCode",
]

train_data = pd.read_csv("train.csv", index_col=0)
test_data = pd.read_csv("test.csv", index_col=0)
ss = pd.read_csv("sample_submission.csv", header=None)

In [3]:
def basic_info(df):
    rows = []
    for col in df.columns:
        rows.append([col, df[col].dtype, df[col].isnull().sum(), len(df[col].unique())])
    return pd.DataFrame(rows, columns=["col", "type", "num_NaN", "val_warm"])

In [4]:
def preprocessing(df, replace_dict=None, ce_dict=None):
    """
    データフレームに対する前処理を行います。

    Parameters:
    - df (pd.DataFrame): 前処理を行うデータフレーム。
    - replace_dict (dict, optional): label encode のための辞書。列名を入れると対応する label encode の数字が得られます。
    - ce_dict (dict, optional): カテゴリカル変数のデータ量を格納する辞書。列名を入れるとそのカテゴリのデータがどのくらいあるかがわかります。

    Returns:
    - pd.DataFrame: 前処理が適用されたデータフレーム。
    - dict: label encode 用の辞書。列名を入れると対応する label encode の数字が得られます。
    - dict: カテゴリカル変数のデータ量を格納する辞書。列名を入れるとそのカテゴリのデータがどのくらいあるかがわかります。
    """
    # Cityは汎用性が低いと考えられるためDrop
    df = df.drop("City", axis=1)

    # Sector, FranchiseCode
    # 32,33→31, 45→44, 49→48に変換
    code_dict = {
        32: 31,
        33: 31,
        45: 44,
        49: 48
    }
    df["Sector"] = df["Sector"].replace(code_dict)

    # RevLineCr, LowDoc
    # YN以外　→ NaN
    revline_dict = {'0': np.nan, 'T': np.nan}
    df["RevLineCr"] = df["RevLineCr"].replace(revline_dict)

    lowdoc_dict = {'C': np.nan, '0': np.nan, 'S': np.nan, 'A': np.nan}
    df["LowDoc"] = df["LowDoc"].replace(lowdoc_dict)

    # DisbursementDate, ApprovalDate
    # 日付型へ変更し年を抽出
    df['DisbursementDate'] = pd.to_datetime(df['DisbursementDate'], format='%d-%b-%y')
    df["DisbursementYear"] = df["DisbursementDate"].dt.year
    df.drop(["DisbursementDate", "ApprovalDate"], axis=1, inplace=True)

    # 本来数値型のものを変換する
    cols = ["DisbursementGross", "GrAppv", "SBA_Appv"]
    df[cols] = df[cols].applymap(lambda x: x.strip().replace('$', '').replace(',', '')).astype(float).astype(int)

    # 特徴量エンジニアリング
    df["FY_Diff"] = df["ApprovalFY"] - df["DisbursementYear"]

    df['SBA_Portion'] = df['SBA_Appv'] / df['GrAppv']
    df["DisbursementGrossRatio"] = df["DisbursementGross"] / df["GrAppv"]
    df["NullCount"] = df.isnull().sum(axis=1)

    # カテゴリカル変数の設定  nanと新規値: -1とする
    df[cat_col] = df[cat_col].fillna(-1)

    # train
    if replace_dict is None:
        # countencode, labelencode
        # ce_dict: 列名を入れるとそのカテゴリのデータがどのくらいあるかを返してくれます
        # replace_dict: 列名を入れるとlabelencodeのための数字を返す
        ce_dict = {}
        replace_dict = {}
        for col in cat_col:
            replace_dict[col] = {}
            vc = df[col].value_counts()
            ce_dict[col] = vc
            replace_dict_in_dict = {}
            for i, k in enumerate(vc.keys()):
                replace_dict_in_dict[k] = i
            replace_dict[col] = replace_dict_in_dict
            df[f"{col}_CountEncode"] = df[col].replace(vc).astype(int)
            df[col] = df[col].replace(replace_dict_in_dict).astype(int)
        return df, replace_dict, ce_dict

    # test
    else:
        for col in cat_col:
            # Count Encode
            test_vals_uniq = df[col].unique()
            ce_dict_in_dict = ce_dict[col]
            for test_val in test_vals_uniq:
                if test_val not in ce_dict_in_dict.keys():
                    ce_dict_in_dict[test_val] = -1
            df[f"{col}_CountEncode"] = df[col].replace(ce_dict_in_dict).astype(int)

            # Label Encode
            test_vals_uniq = df[col].unique()
            replace_dict_in_dict = replace_dict[col]
            for test_val in test_vals_uniq:
                if test_val not in replace_dict_in_dict.keys():
                    replace_dict_in_dict[test_val] = -1
            df[col] = df[col].replace(replace_dict_in_dict).astype(int)
        return df

In [5]:
def f1_optimization(val_y, preds_y_proba):
    """
    F1-scoreを最適化するための閾値を見つける関数。

    Parameters:
    - val_y (array-like): 真のラベル値。
    - preds_y_proba (array-like): モデルの予測確率。

    Returns:
    - float: 最大の平均F1スコア。
    - float: 最適な閾値。
    """
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]


In [6]:
def model_rf(X_train, y_train, cat_col):
    list_models = []
    list_metrics_auc = []
    list_metrics_f1 = []
    list_cutoff = []
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
    
    for fold, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train), start=1):
        trn_x = X_train.iloc[trn_idx, :]
        trn_y = y_train[trn_idx]
        val_x = X_train.iloc[val_idx, :]
        val_y = y_train[val_idx]

        model_rf = RandomForestClassifier(n_estimators=100, random_state=0)
        model_rf.fit(trn_x, trn_y)
        list_models.append(model_rf)

        preds_y_proba = model_rf.predict_proba(val_x)[:, 1]
        auc = roc_auc_score(val_y, preds_y_proba)
        f1, threshold = f1_optimization(val_y, preds_y_proba)

        list_metrics_auc.append(auc)
        list_metrics_f1.append(f1)
        list_cutoff.append(threshold)

        print(f"Fold: {fold}, AUC: {auc}, f1 score: {f1} Threshold: {threshold}")

    print(np.mean(list_metrics_auc), np.mean(list_metrics_f1), np.median(list_cutoff))
    return list_models, list_metrics_auc, list_metrics_f1, list_cutoff

In [7]:
def predict_test_rf(test_data, list_models ,list_cutoff):
    threshold = np.median(list_cutoff)
    preds_y_proba = np.zeros(len(test_data))
    
    for model in list_models:
        preds_y_proba += model.predict_proba(test_data)[:, 1] / len(list_models)
    
    y_preds = [1 if prob > threshold else 0 for prob in preds_y_proba]
    return y_preds

In [8]:
def MIS_Status_corr_confirm(df, y_col):
    """
    MIS_statusとの相関を確認
    
    Returns:
    - pd.DataFrame: 各変数のMIS_stautsとの相関係数を平均値でソート(ピアソンとスピアマンでそれぞれ相関を見る)
    """
    s_per = df.corr("pearson")[y_col].sort_values()
    s_spr = df.corr("spearman")[y_col].sort_values()
    df_corr = pd.concat([s_per, s_spr], axis=1)
    df_corr.columns = ["Pearson", "Spearman"]

    # 平均値でソート
    return df_corr.loc[df_corr.mean(axis=1).sort_values(ascending=False).keys(), :].drop(y_col)

In [9]:
def sumbmit_score(ss, y_pred):
    """
    予測値をコンペ提出用のファイルに代入させてsumbimitファイルをcsvで保存
    
    Parameters:
    - ss (pd.DataFrame): 提出用のサンプルファイル。通常、1列目は識別子やインデックス、2列目は予測値が格納されている。
    - y_pred (array-like): モデルの予測結果。

    Returns:
    - None: 保存が成功した場合は何も返さない。
    """
    ss[1] = y_pred
    ss[1] = ss[1].astype(int)
    print(ss[1].value_counts())
    ss.to_csv("submit_rf.csv", header=False, index=False)

In [10]:
def get_feature_importance_rf_permutation(X_train, y_train, list_models, feature_names):
    feature_importance_df = pd.DataFrame()

    for i, model in enumerate(list_models, start=1):
        result = permutation_importance(model, X_train, y_train, n_repeats=10, random_state=0, n_jobs=-1)
        fold_importance_df = pd.DataFrame()
        fold_importance_df["Feature"] = feature_names
        fold_importance_df["Importance"] = result.importances_mean
        fold_importance_df["Fold"] = i
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    # 特徴量ごとにグループ化し、平均重要度を計算
    mean_importance = feature_importance_df.groupby("Feature")["Importance"].mean().reset_index()

    # 平均重要度で降順にソート
    mean_importance = mean_importance.sort_values(by="Importance", ascending=False)

    return mean_importance

In [11]:
# train_data.head()

In [12]:
basic_info(train_data)

,col,type,num_NaN,val_warm
0,Term,int64,0,228
1,NoEmp,int64,0,196
2,NewExist,float64,0,2
3,CreateJob,int64,0,49
4,RetainedJob,int64,0,83
5,FranchiseCode,int64,0,271
6,RevLineCr,object,1079,5
7,LowDoc,object,531,7
8,DisbursementDate,object,150,917
9,MIS_Status,int64,0,2


In [13]:
train_data, replace_dict, ce_dict = preprocessing(train_data)
test_data = preprocessing(test_data, replace_dict=replace_dict, ce_dict=ce_dict)

In [14]:
# test_data.head()

In [15]:
basic_info(train_data)

,col,type,num_NaN,val_warm
0,Term,int64,0,228
1,NoEmp,int64,0,196
2,NewExist,float64,0,2
3,CreateJob,int64,0,49
4,RetainedJob,int64,0,83
5,FranchiseCode,int64,0,271
6,RevLineCr,int64,0,3
7,LowDoc,int64,0,3
8,MIS_Status,int64,0,2
9,Sector,int64,0,20


In [16]:
MIS_Status_corr_confirm(train_data, y_col)

,Pearson,Spearman
RevLineCr_CountEncode,0.138850,0.127572
NoEmp,0.092940,0.171855
Term,0.122125,0.119292
LowDoc_CountEncode,0.107665,0.114348
Sector_CountEncode,0.104510,0.105012
DisbursementGrossRatio,0.047301,0.032333
FY_Diff,0.034412,0.042713
State,0.024825,0.026830
BankState,0.009405,0.005933
DisbursementGross,0.000481,0.000447


In [17]:
X_train = train_data.drop(y_col, axis=1)
y_train = train_data[y_col]

In [18]:
list_models_rf, list_metrics_auc_rf, list_metrics_f1_rf, list_cutoff_rf = model_rf(X_train, y_train, cat_col)

Fold: 1, AUC: 0.7519141913013951, f1 score: 0.6636134922984956 Threshold: 0.7811246420984449
Fold: 2, AUC: 0.7640849497906697, f1 score: 0.6754347154564649 Threshold: 0.7786162669244053
Fold: 3, AUC: 0.7634309151754248, f1 score: 0.6704063448470271 Threshold: 0.7561127026155161
0.75981001875583 0.6698181842006625 0.7786162669244053


In [19]:
y_preds_rf = predict_test_rf(test_data, list_models_rf, list_cutoff_rf)

In [20]:
sumbmit_score(ss, y_preds_rf)

1    38711
0     3597
Name: 1, dtype: int64


In [21]:
feature_importance_rf = get_feature_importance_rf_permutation(X_train, y_train, list_models_rf, X_train.columns)
feature_importance_rf

,Feature,Importance
26,UrbanRural,0.012585
14,NoEmp,0.010543
25,Term,0.008181
18,RevLineCr_CountEncode,0.007085
17,RevLineCr,0.006833
11,LowDoc,0.006749
12,LowDoc_CountEncode,0.006304
6,DisbursementYear,0.006157
16,RetainedJob,0.005675
15,NullCount,0.005140


In [22]:
%run model_rf.py

Fold: 1, AUC: 0.7519141913013951, f1 score: 0.6636134922984956 Threshold: 0.7811246420984449
Fold: 2, AUC: 0.7640849497906697, f1 score: 0.6754347154564649 Threshold: 0.7786162669244053
Fold: 3, AUC: 0.7634309151754248, f1 score: 0.6704063448470271 Threshold: 0.7561127026155161
0.75981001875583 0.6698181842006625 0.7786162669244053
1    38711
0     3597
Name: 1, dtype: int64


,Feature,Importance
26,UrbanRural,0.012585
14,NoEmp,0.010543
25,Term,0.008181
18,RevLineCr_CountEncode,0.007085
17,RevLineCr,0.006833
11,LowDoc,0.006749
12,LowDoc_CountEncode,0.006304
6,DisbursementYear,0.006157
16,RetainedJob,0.005675
15,NullCount,0.005140
